In [6]:
import rasterio as rio
from rasterio.transform import rowcol
from rasterio.warp import transform_bounds
from rasterio.transform import from_origin
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import numpy as np
from netCDF4 import Dataset
from netCDF4 import num2date
import netCDF4 as nc
import json

### Enrichment of coordinats with or without timestamps
This notebook provides examples of how to extract information for on coordinats with or without timestamps

#### 1. Simple coordinates
The example will use dummy data generated by the function *simple_dummy_data*. That function generates a pandas dataframe for a set number of subjects that come from a area defined by a bounding box.  
The function *link_coordinates* will do the job. The result will be a dictionary there the key is a tuple of subjectIDs and the value corresponding to the key is the list of the variable of interest. The idea of this data structure is based on the fact that climate and atmosphere data is not of high resolution e.g. in the case of uk biobank the amount of subject coming from the London ares is high and it's not unlikely hat they share the same climate or atmosphere data.  

In [7]:
def simple_dummy_data(number_of_subjects, bounding_box):
    min_lat, min_lon, max_lat, max_lon = bounding_box
    latitudes = np.random.uniform(low=min_lat, high=max_lat, size=number_of_subjects)
    longitudes = np.random.uniform(low=min_lon, high=max_lon, size=number_of_subjects)
    # Create a DataFrame
    df = pd.DataFrame({
        'latitude': latitudes,
        'longitude': longitudes
    })
    # Create a GeoDataFrame
    geometry = [Point(lon, lat) for lon, lat in zip(df['longitude'], df['latitude'])]
    gdf = gpd.GeoDataFrame(df, geometry=geometry)
    gdf.drop(columns=['longitude','latitude'],inplace=True)
    # start_date.value is ni ns and start_date.value // 10**9 in seconds
    gdf['subjectID'] = np.random.permutation(len(gdf))
    return gdf


def link_coordinates(geocoordinates_gdf, file_path):
    # Initialize an empty dictionary to store the results
    rowcol_dict = {}
    result_dict = {}
    bbox = geocoordinates_gdf.geometry.total_bounds
    with rio.open(file_path) as src:
        for index, row in geocoordinates_gdf.iterrows():
            point = row.geometry
            row_idx, col_idx = rowcol(src.transform, point.x, point.y)
            # Use the row_idx and col_idx as a key
            key = (row_idx, col_idx)     
            # Append the index to the list corresponding to the key
            if key not in rowcol_dict:
                rowcol_dict[key] = []
            rowcol_dict[key].append(index)
        row_start, col_start = rowcol(src.transform, bbox[0], bbox[1])
        row_stop, col_stop = rowcol(src.transform, bbox[2], bbox[3])
        # I tried also to load all or only the loactaion of interest
        # using the bounding box seems was the fastest
        # mybe later cluster the points and set different bounding boxes
        data = src.read(window=((row_start, row_stop+1), (col_start, col_stop+1)))
        for k,v in rowcol_dict.items():
            result_dict[tuple(v)] = data[:, k[0]-row_start, k[1]-col_start].flatten()
    return result_dict


In [8]:
# create a dummy dataset with sunjectIDs and a ranom location from a fixed bounding box 
number_of_subjects = 100
berlin = [52.7136,12.9673,52.2839,13.816]
dummy = simple_dummy_data(number_of_subjects, berlin)
print(dummy.head(5))

                    geometry  subjectID
0  POINT (13.15265 52.63381)          7
1  POINT (13.49876 52.41996)         61
2  POINT (12.98712 52.43991)         81
3  POINT (13.18040 52.36639)         80
4  POINT (13.35099 52.30577)         74


##### Link the dummy locations to a netCDF file.

result contains a dictonary there the keys are the tuples of subjectIDs and the corresponding values are extracted from the netCDF file

In [16]:
file_path = 'test_data\\black_carbon_aerosol_optical_depth_550nm\\2021.nc'
result = link_coordinates(dummy, file_path)
print(result)


{(0, 1, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 34, 36, 37, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 52, 53, 55, 56, 57, 58, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 72, 73, 74, 75, 76, 77, 78, 80, 83, 84, 85, 87, 88, 90, 95, 96, 97, 98, 99): array([1.82338292e-03, 1.89317437e-03, 7.12050707e-04, 7.14481459e-04,
       1.18279981e-03, 2.06811307e-03, 3.00285779e-03, 3.15961731e-03,
       1.93148456e-03, 3.09452438e-03, 1.00057269e-03, 2.11407489e-04,
       7.47761515e-04, 8.27804673e-04, 1.59886421e-03, 1.59236998e-03,
       1.44353462e-03, 1.34349801e-03, 2.42756819e-03, 1.36618898e-03,
       7.78171350e-04, 1.31263165e-03, 9.91489971e-04, 8.73857818e-04,
       1.24161562e-03, 2.95889168e-03, 1.43849384e-03, 3.02748493e-04,
       9.11783543e-04, 1.69238483e-03, 8.71864264e-04, 1.18796260e-03,
       2.67200428e-03, 1.52607285e-03, 5.21947630e-04, 1.74184074e-03,
       2.95742578e-03, 3.80682154e-03, 1.04003318e

#### 2. Linking ccordinates with timestamps
The example will use dummy data generated by the function *dummy_data_with_timestamos*. That function generates a pandas dataframe for a set number of subjects that come from a area defined by a bounding box. Each row contains a subjectID, coordinates and a timestamp. There can be multiple rows with the same subjectID with different timestamps.    
The function *link_coordinates* will do the job. The result will be a dictonary there the key is a tuple of subjectIDs and the value corresponding tothe key is the list of the varibles of interest. The idea of this data structure is based on the fact that climate and atmosphere data is not of high resolution e.g. in the case of uk biobank the amount of subject coming from the London ares is high and it's not unlikely hat they share the same climate or atmosphere data. 

In [17]:
def dummy_data_with_timestamps(number_of_subjects, number_of_observations, bounding_box, start_date, end_date):
    min_lat, min_lon, max_lat, max_lon = bounding_box
    latitudes = np.random.uniform(low=min_lat, high=max_lat, size=number_of_observations)
    longitudes = np.random.uniform(low=min_lon, high=max_lon, size=number_of_observations)
    
    # Create a DataFrame
    df = pd.DataFrame({
        'latitude': latitudes,
        'longitude': longitudes
    })
    
    # Create a GeoDataFrame
    geometry = [Point(lon, lat) for lon, lat in zip(df['longitude'], df['latitude'])]
    gdf = gpd.GeoDataFrame(df, geometry=geometry)
    gdf.drop(columns=['longitude','latitude'],inplace=True)
    # start_date.value is ni ns and start_date.value // 10**9 in seconds
    gdf['date'] = pd.to_datetime(np.random.randint(start_date.value // 10**9, end_date.value // 10**9, len(gdf)), unit='s')
    gdf['subjectID'] = np.random.randint(1, number_of_subjects+1, len(gdf))
    return gdf


def link_coordinates(geocoordinates_gdf, file_path):
    # Initialize an empty dictionary to store the results
    rowcol_dict = {}
    result_dict = {}
    bbox = geocoordinates_gdf.geometry.total_bounds
    with rio.open(file_path) as src:
        dataset = nc.Dataset(file_path)
        times = dataset.variables['valid_time']
        times = nc.num2date(times[:], times.units)
        for index, row in geocoordinates_gdf.iterrows():
            point = row.geometry
            row_idx, col_idx = rowcol(src.transform, point.x, point.y)
            time_idx = np.argmin([np.abs(row.date - t) for t in times])
            # Use the timestamp, row_idx and col_idx as a key
            key = (row_idx, col_idx, time_idx)     
            # Append the index to the list corresponding to the key
            if key not in rowcol_dict:
                rowcol_dict[key] = []
            rowcol_dict[key].append(index)
        # adjust the indices to the bounding box
        row_start, col_start = rowcol(src.transform, bbox[0], bbox[1])
        row_stop, col_stop = rowcol(src.transform, bbox[2], bbox[3])
        # I tried also to load all or only the loactaion of interest
        # using the bounding box seems was the fastest
        # mybe later cluster the points and set different bounding boxes
        data = src.read(window=((row_start, row_stop+1), (col_start, col_stop+1)))
        for k,v in rowcol_dict.items():
            v.append(row.date.strftime('%Y-%m-%d %H:%M:%S'))
            result_dict[tuple(v)] = data[k[2], k[0]-row_start, k[1]-col_start].flatten()
    return result_dict


In [19]:
number_of_subjects = 100 
number_of_observations = 1000
berlin = [52.7136,12.9673,52.2839,13.816]
start_date = pd.to_datetime('2021-01-01')
end_date = pd.to_datetime('2021-12-31')
dummy = dummy_data_with_timestamps(number_of_subjects, number_of_observations, berlin, start_date, end_date)
print(dummy.head(5))

                    geometry                date  subjectID
0  POINT (13.47945 52.50050) 2021-07-09 20:58:23         92
1  POINT (13.75775 52.52421) 2021-01-28 16:47:13         12
2  POINT (13.68935 52.57970) 2021-12-30 22:05:34         44
3  POINT (13.56685 52.59000) 2021-08-13 20:48:00         12
4  POINT (12.96882 52.53555) 2021-10-20 02:13:48         23


In [24]:
# data element in the netcdf file
dataset = nc.Dataset(file_path) 
# Use 'valid_time' as the time variable
print(dataset.variables)
times = dataset.variables['valid_time']
times = nc.num2date(times[:], times.units)
print(times)


{'valid_time': <class 'netCDF4._netCDF4.Variable'>
int64 valid_time(valid_time)
    long_name: time
    standard_name: time
    units: seconds since 1970-01-01
    calendar: proleptic_gregorian
unlimited dimensions: 
current shape = (365,)
filling off, 'latitude': <class 'netCDF4._netCDF4.Variable'>
float64 latitude(latitude)
    _FillValue: nan
    units: degrees_north
    standard_name: latitude
    long_name: latitude
    stored_direction: decreasing
unlimited dimensions: 
current shape = (61,)
filling on, 'longitude': <class 'netCDF4._netCDF4.Variable'>
float64 longitude(longitude)
    _FillValue: nan
    units: degrees_east
    standard_name: longitude
    long_name: longitude
unlimited dimensions: 
current shape = (77,)
filling on, 'bcaod550': <class 'netCDF4._netCDF4.Variable'>
float32 bcaod550(valid_time, latitude, longitude)
    _FillValue: nan
    GRIB_paramId: 210211
    GRIB_dataType: an
    GRIB_numberOfPoints: 4697
    GRIB_typeOfLevel: surface
    GRIB_stepUnits: 1
    G

##### Link the dummy locations with date to a netCDF file.

result contains a dictonary there the keys are the tuples of subjectIDs and the corresponding values are extracted from the netCDF file

In [26]:
file_path = 'test_data\\black_carbon_aerosol_optical_depth_550nm\\2021.nc'

print(link_coordinates(dummy, file_path))

{(0, 34, 607, 927, '2021-09-20 17:16:30'): array([0.00654066], dtype=float32), (1, 88, 550, 885, '2021-09-20 17:16:30'): array([0.00030275], dtype=float32), (2, 61, 244, 418, 697, 792, '2021-09-20 17:16:30'): array([0.000987], dtype=float32), (3, 416, 451, 504, 534, '2021-09-20 17:16:30'): array([0.02358796], dtype=float32), (4, 850, '2021-09-20 17:16:30'): array([0.00405802], dtype=float32), (5, 981, '2021-09-20 17:16:30'): array([0.01074397], dtype=float32), (6, 190, 211, 554, '2021-09-20 17:16:30'): array([0.00125378], dtype=float32), (7, '2021-09-20 17:16:30'): array([0.01343931], dtype=float32), (8, 147, '2021-09-20 17:16:30'): array([0.03226109], dtype=float32), (9, 165, '2021-09-20 17:16:30'): array([0.00305155], dtype=float32), (10, 110, 262, 468, 568, 921, '2021-09-20 17:16:30'): array([0.01718817], dtype=float32), (11, '2021-09-20 17:16:30'): array([5.40974e-05], dtype=float32), (12, 255, 470, '2021-09-20 17:16:30'): array([0.01354233], dtype=float32), (13, 401, 626, 823, '20

In [4]:
!pip install rasterio
!pip install numpy
!pip install geopandas
!pip install pandas
!pip install shapely
!pip install imageio-ffmpeg
!pip install rasterio
!pip install netCDF4
!pip install geopandas
!pip install xarray
!pip install rioxarray

  Using cached rasterio-1.4.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.1 kB)
  Using cached affine-2.4.0-py3-none-any.whl.metadata (4.0 kB)
  Using cached cligj-0.7.2-py3-none-any.whl.metadata (5.0 kB)
  Using cached click_plugins-1.1.1-py2.py3-none-any.whl.metadata (6.4 kB)
Using cached rasterio-1.4.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (22.2 MB)
Using cached cligj-0.7.2-py3-none-any.whl (7.1 kB)
Using cached affine-2.4.0-py3-none-any.whl (15 kB)
Using cached click_plugins-1.1.1-py2.py3-none-any.whl (7.5 kB)
  Using cached geopandas-1.0.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached pyogrio-0.10.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (5.5 kB)
  Using cached pyproj-3.7.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (31 kB)
Using cached geopandas-1.0.1-py3-none-any.whl (323 kB)
Using cached pyogrio-0.10.0-cp311-cp311-manylinux_2_28_x86_64.whl (24.1 MB)
Using cached pyproj-3.7.0-cp311-cp311-manylin